In [27]:
import ccxt
import pandas as pd
from IPython.display import display, HTML

# Function to fetch ETH/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)
    
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to create the pivot table with positive percentages in green
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('unique_week_number')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='year', columns='unique_week_number', aggfunc='mean')

    # Calculate mean, median, max, and min for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)

    # Add the average, median as the last rows
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value

    return pivot_table

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ETH/USDT'
    ETH_data = fetch_data(symbol)

    # Calculate weekly return
    ETH_data = calculate_weekly_return(ETH_data)

    # Step 1: Filter 2024 Data
    ETH_2024 = ETH_data[ETH_data['year'] == 2024]
    # filter 2024 data and get only 1月-5月还有11月-12月
    ETH_2024 = ETH_2024[(ETH_2024['month'] >= 1) & (ETH_2024['month'] <= 4) | (ETH_2024['month'] >= 11) & (ETH_2024['month'] <= 12)]

    # Step 2: Analyze Historical Data (before 2024) for Buy Signals
    historical_data = ETH_data[ETH_data['year'] < 2024]

    # Create pivot table for historical data
    historical_pivot = create_pivot_table(historical_data, 'weekly_return')
    print("Historical Pivot Table:")
    display(historical_pivot)

    # Extract Avg and Median for each column (historical)
    historical_pivot_avg = historical_pivot.loc['Avg']
    historical_pivot_median = historical_pivot.loc['Median']

    print("\nHistorical Avg:")
    display(historical_pivot_avg)
    print("\nHistorical Median:")
    display(historical_pivot_median)


    # Step 3: Identify Buy Signals (when Avg and Median are both positive)
    buy_signals = historical_pivot_avg[(historical_pivot_avg > 0) & (historical_pivot_median > 0)].index

    # Step 4: Calculate Win Rate for Buy Signals
    # Filter 2024 data where the week belongs to buy signals and check if the return is positive
    ETH_2024['win'] = ETH_2024['unique_week_number'].apply(
        lambda x: 1 if x in buy_signals and ETH_2024.loc[ETH_2024['unique_week_number'] == x, 'weekly_return'].values[0] > 0 else 0
    )

    # Step 5: Calculate total buy signal weeks and winning weeks
    total_buy_signal_weeks = len(ETH_2024[ETH_2024['unique_week_number'].isin(buy_signals)])
    winning_weeks = ETH_2024[ETH_2024['win'] == 1].shape[0]
    total_profit_of_winning_weeks = ETH_2024[ETH_2024['win'] == 1]['weekly_return'].sum()
    # total_loss_of_losing_weeks = win=0 and have buy signal
    total_loss_of_losing_weeks = ETH_2024[(ETH_2024['win'] == 0) & (ETH_2024['unique_week_number'].isin(buy_signals))]['weekly_return'].sum()

    # Step 6: Calculate Win Rate
    if total_buy_signal_weeks > 0:
        win_rate = (winning_weeks / total_buy_signal_weeks) * 100
    else:
        win_rate = 0.0

    # Merge the historical pivot data (Avg and Median) with the 2024 data
    # Add Avg and Median values to the 2024 DataFrame by aligning on unique_week_number
    ETH_2024['Avg'] = ETH_2024['unique_week_number'].map(historical_pivot_avg.to_dict())
    ETH_2024['Median'] = ETH_2024['unique_week_number'].map(historical_pivot_median.to_dict())


    # Display the results with Avg and Median included
    # display(ETH_2024)

    # Output the numbers for total buy signal weeks and winning weeks
    print(f"Total buy signal weeks: {total_buy_signal_weeks}")
    print(f"Total winning weeks: {winning_weeks}")
    print(f"Win rate for buy signals from previous years: {win_rate:.2f}%")
    print(f"Total profit of winning weeks: {total_profit_of_winning_weeks:.2f}%")
    print(f"Total loss of losing weeks: {total_loss_of_losing_weeks:.2f}%")

# Run the main function
main()


Historical Pivot Table:


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,...,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,
2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.404587,16.496711,30.322399,-0.548786,NaN,-7.158375,65.587926,-7.012238,11.310282,NaN
2018,53.216191,19.936960,-22.180432,19.347950,-34.218346,-1.534810,12.666650,-7.663008,2.994795,-16.819561,...,0.193278,-15.818438,-33.718722,-1.753055,NaN,-20.037726,-9.210808,52.373642,6.773619,11.646470
2019,-25.817142,2.890427,-4.686968,-4.177080,NaN,16.575049,6.897379,-1.248411,-1.135503,3.652374,...,4.092987,-2.720152,-23.860298,7.671977,NaN,-0.185775,-5.304440,-7.279236,1.695428,0.751712
2020,8.251459,13.803221,0.641525,12.226788,NaN,21.227716,13.267405,6.021863,-20.652095,-8.438547,...,14.525562,-1.366968,24.987167,2.905202,4.376041,-1.810203,8.017806,7.367184,42.793753,NaN
2021,28.203163,-1.737307,12.982556,-5.720484,NaN,22.794773,11.726012,7.416543,-26.601984,21.674526,...,7.582776,0.301566,-7.958284,0.823532,-2.289529,-1.583247,-4.984504,3.552880,-5.790475,NaN
2022,-17.662502,6.183752,-24.138230,2.468688,17.424193,-6.018658,-8.677738,-0.198323,-2.503067,-1.397327,...,-22.111271,-6.491199,4.523687,7.164037,NaN,-1.281069,-6.329375,2.996467,-1.491974,NaN
2023,7.483775,20.336395,4.825703,1.062411,-0.955172,-7.009165,10.887762,-2.284877,-4.678409,1.007306,...,8.086335,-1.624696,2.528499,6.333097,NaN,7.270082,-6.626027,3.073939,0.787530,NaN
Avg,8.945824,10.235574,-5.425974,4.201379,-5.916442,7.672484,7.794578,0.340631,-8.762710,-0.053538,...,2.253465,-1.603311,-0.453650,3.228000,1.043256,-3.540902,5.878654,7.867520,8.011166,6.199091
Median,7.867617,9.993486,-2.022722,1.765550,-0.955172,7.520119,11.306887,-0.723367,-3.590738,-0.195011,...,4.092987,-1.624696,2.528499,2.905202,1.043256,-1.583247,-5.304440,3.073939,1.695428,6.199091



Historical Avg:


unique_week_number
A-M1-W1      8.945824
A-M1-W2     10.235574
A-M1-W3     -5.425974
A-M1-W4      4.201379
A-M1-W5     -5.916442
B-M2-W1      7.672484
B-M2-W2      7.794578
B-M2-W3      0.340631
B-M2-W4     -8.762710
C-M3-W1     -0.053538
C-M3-W2     -4.591563
C-M3-W3      0.943264
C-M3-W4     -3.242945
C-M3-W5     18.829336
D-M4-W1      6.380388
D-M4-W2      9.000536
D-M4-W3      2.701331
D-M4-W4      5.917902
D-M4-W5      9.131015
E-M5-W1      3.130785
E-M5-W2      3.194161
E-M5-W3    -11.291132
E-M5-W4      5.737918
E-M5-W5      4.094533
F-M6-W1     -9.865144
F-M6-W2     -4.723877
F-M6-W3      1.645076
F-M6-W4      0.326445
F-M6-W5      1.187247
G-M7-W1      2.576461
G-M7-W2     -4.802334
G-M7-W3     10.823951
G-M7-W4      5.976144
G-M7-W5     -3.072973
H-M8-W1     -0.026742
H-M8-W2      1.374424
H-M8-W3     -3.706235
H-M8-W4     -0.913410
H-M8-W5      5.162324
I-M9-W1     -5.768842
I-M9-W2     -2.964612
I-M9-W3      2.133038
I-M9-W4      0.247306
I-M9-W5      0.496278
J-M10-W1     


Historical Median:


unique_week_number
A-M1-W1      7.867617
A-M1-W2      9.993486
A-M1-W3     -2.022722
A-M1-W4      1.765550
A-M1-W5     -0.955172
B-M2-W1      7.520119
B-M2-W2     11.306887
B-M2-W3     -0.723367
B-M2-W4     -3.590738
C-M3-W1     -0.195011
C-M3-W2      4.741697
C-M3-W3     -1.535984
C-M3-W4      1.476760
C-M3-W5     18.829336
D-M4-W1      4.704339
D-M4-W2      8.727620
D-M4-W3      2.310089
D-M4-W4      3.375691
D-M4-W5      9.131015
E-M5-W1     -4.096946
E-M5-W2     -2.789374
E-M5-W3     -4.099272
E-M5-W4      7.152558
E-M5-W5     -0.408571
F-M6-W1    -10.781333
F-M6-W2     -5.237682
F-M6-W3      2.270285
F-M6-W4     -0.920771
F-M6-W5      1.187247
G-M7-W1      5.370589
G-M7-W2     -4.530417
G-M7-W3      8.734154
G-M7-W4      3.492763
G-M7-W5     -1.692457
H-M8-W1      0.943057
H-M8-W2     -0.674127
H-M8-W3     -4.003705
H-M8-W4     -1.356184
H-M8-W5     10.689955
I-M9-W1     -1.126027
I-M9-W2      0.312844
I-M9-W3     -2.574454
I-M9-W4     -1.382645
I-M9-W5      0.496278
J-M10-W1     

Total buy signal weeks: 12
Total winning weeks: 6
Win rate for buy signals from previous years: 50.00%
Total profit of winning weeks: 79.23%
Total loss of losing weeks: -28.82%


In [1]:
import ccxt
import pandas as pd
from IPython.display import display, HTML

# Function to fetch ETH/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    #delete all 2024 data
    #df = df[df['year'] < 2024]
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    # Map months to letters (A = Jan, B = Feb, ..., L = Dec)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    # Create unique "week number" format: MonthLetter-MonthNumber-WeekOfMonth
    

    
    # Create unique "week number" format: MonthLetter-WeekOfMonth
    # df['unique_week_number'] = df['month_letter'] + '-W' + df['week_of_month'].astype(str)
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)
    return df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to create and style the pivot table with positive percentages in green
# Function to create and style the pivot table with positive percentages in green
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('unique_week_number')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='year', columns='unique_week_number', aggfunc='mean')

    # Calculate mean, median, max, and min for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)
    max_value = pivot_table.max(axis=0)
    min_value = pivot_table.min(axis=0)

    # Add the average, median, max, and min as the last rows
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value
    pivot_table.loc['Max'] = max_value
    pivot_table.loc['Min'] = min_value

    # Function to color positive values green and negative values red
    def color_positive_negative(val):
        if isinstance(val, (int, float)):
            if val > 0:
                return 'background-color: lightgreen; color: white;'  # Green for positive values
            elif val < 0:
                return 'background-color: pink; color: white;'  # Red for negative values
        return ''  # No change for non-numeric values

    # Style the table
    styled_table = pivot_table.style \
        .format('{:.2f}%') \
        .applymap(color_positive_negative)  # Apply the coloring function to the table
    
    # Set additional styles
    styled_table.set_table_styles([ 
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%'), ('display', 'block'), ('overflow-x', 'auto'), ('position', 'relative')]},  # Set relative position for the table container
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
        
        # Add a line below the "Avg" row to separate it from the rest of the table
        {'selector': 'tr:nth-last-child(5)', 'props': [('border-bottom', '2px solid black')]},  # Row for "Avg"
        
        # # Freeze the first column (Year) and the header row
        # {'selector': 'thead', 'props': [('position', 'sticky'), ('top', '0'), ('z-index', '1')]},  # Freeze header row
        # {'selector': 'thead th:first-child', 'props': [('position', 'sticky'), ('left', '0'), ('background-color', '#f2f2f2'), ('z-index', '2')]},  # Freeze the first column in header
        # {'selector': 'tbody td:first-child', 'props': [('position', 'sticky'), ('left', '0'), ('background-color', '#f2f2f2'), ('z-index', '2')]},  # Freeze the first column in body
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table



# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ETH/USDT'
    ETH_data = fetch_data(symbol)

    # Calculate indicators
    ETH_data = calculate_max_drawdown(ETH_data)
    ETH_data = calculate_high_to_open_change(ETH_data)
    ETH_data = calculate_weekly_return(ETH_data)

    # Create and display the pivot tables
    styled_table_max_drawdown = create_pivot_table(ETH_data, 'max_drawdown')
    display_table("ETH/USDT Weekly Max Drawdown (Month-Letter-WeekOfMonth)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(ETH_data, 'high_to_open_change')
    display_table("ETH/USDT Weekly Open to High % Change (Month-Letter-WeekOfMonth)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(ETH_data, 'weekly_return')
    display_table("ETH/USDT Weekly Returns (Open to Close, Month-Letter-WeekOfMonth)", styled_table_weekly_return)

# Run the main function
main()


C:\Users\nicol\AppData\Local\Temp\ipykernel_30724\3674096426.py:84: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-7.68%,-51.79%,-8.05%,nan%,-30.58%,-34.58%,-2.11%,-2.84%,nan%,-6.83%,-7.87%,-15.65%,-7.35%,-9.92%,-4.80%,-0.10%,-1.44%,-16.82%,nan%,-18.53%,0.00%,-28.64%,-9.01%,nan%
2018,-2.21%,-18.97%,-43.99%,-13.44%,-37.40%,-30.61%,-0.14%,-13.72%,-1.01%,-26.15%,-37.59%,-5.71%,-31.88%,nan%,-4.24%,-3.32%,-6.85%,-4.32%,-9.16%,-20.28%,-10.02%,-23.20%,-11.33%,nan%,-19.13%,-14.41%,-15.61%,-10.96%,nan%,-1.76%,-14.31%,-2.18%,-2.26%,-14.96%,-25.52%,-21.37%,-13.30%,-0.94%,nan%,-37.39%,-14.69%,-12.64%,-16.74%,nan%,-7.09%,-16.46%,-1.14%,-2.56%,-5.84%,-2.13%,-19.57%,-42.50%,-13.73%,nan%,-29.63%,-12.30%,-0.33%,-13.80%,-6.61%
2019,-27.31%,-0.20%,-5.55%,-9.93%,nan%,-5.70%,-5.55%,-1.55%,-3.98%,-6.13%,-5.06%,-5.38%,-3.32%,nan%,-1.30%,-9.28%,-6.88%,-10.82%,-3.71%,-3.37%,-1.46%,-10.90%,-9.16%,nan%,-15.74%,-1.73%,-2.35%,-9.21%,nan%,-6.91%,-26.88%,-15.05%,-12.08%,-3.39%,-8.65%,-17.73%,-7.61%,-12.29%,nan%,-3.36%,-4.51%,-0.40%,-28.96%,-2.51%,-0.82%,-6.81%,-12.40%,-3.71%,nan%,-0.64%,-5.97%,-24.93%,-6.10%,nan%,-5.02%,-7.44%,-18.39%,-7.73%,-6.31%
2020,-0.38%,-2.93%,-6.74%,-1.60%,nan%,-2.01%,-5.32%,-6.52%,-23.50%,-10.24%,-56.88%,-18.35%,-2.30%,-0.33%,-0.06%,-6.45%,-7.40%,-4.10%,nan%,-14.42%,-6.19%,-7.40%,-1.50%,nan%,-2.83%,-7.51%,-5.81%,-5.28%,-1.61%,-0.27%,-5.25%,-2.12%,-1.66%,nan%,-2.58%,-6.23%,-12.38%,-5.30%,-28.10%,-8.46%,-3.12%,-15.56%,-6.53%,nan%,-5.72%,-3.47%,-3.40%,-8.02%,nan%,-6.67%,-4.67%,-0.58%,-14.11%,-2.83%,-11.83%,-2.57%,-13.61%,-0.59%,nan%
2021,-9.03%,-27.05%,-15.43%,-13.30%,nan%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,nan%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-24.20%,-1.10%,nan%,-11.89%,-13.83%,-9.81%,-1.80%,nan%,-4.43%,-3.94%,-10.80%,-5.70%,-2.50%,-23.93%,-8.69%,-20.33%,-9.07%,nan%,-4.32%,-1.27%,-4.45%,-4.71%,nan%,-3.29%,-3.27%,-14.48%,-8.14%,-18.42%,-8.77%,-11.77%,-4.36%,-11.78%,nan%
2022,-21.63%,-7.08%,-31.28%,-14.97%,-4.83%,-7.15%,-10.28%,-12.28%,-3.13%,-4.17%,-0.68%,-1.10%,-2.58%,nan%,-10.75%,-7.90%,-3.51%,-7.25%,nan%,-12.06%,-28.56%,-11.29%,-16.65%,-4.23%,-21.22%,-38.56%,-7.52%,-16.68%,nan%,-2.69%,-13.87%,-0.19%,-15.12%,nan%,-7.08%,-2.55%,-21.27%,-12.00%,-0.33%,-5.61%,-24.95%,-8.58%,-3.20%,nan%,-1.07%,-10.06%,-4.08%,-2.93%,-5.54%,-31.55%,-7.27%,-5.88%,-3.59%,nan%,-4.80%,-8.53%,-2.75%,-3.07%,nan%
2023,-0.61%,-0.37%,-3.29%,-6.72%,-6.67%,-8.53%,-3.49%,-7.29%,-5.91%,-12.54%,-0.97%,-3.96%,-4.95%,nan%,-1.77%,-0.67%,-13.86%,-4.14%,nan%,-3.42%,-6.97%,-1.56%,-2.41%,-3.60%,-9.26%,-7.23%,-1.33%,-4.33%,nan%,-5.75%,-0.87%,-3.69%,-2.94%,-2.58%,-1.55%,-15.79%,-6.24%,-3.38%,nan%,-2.19%,-5.34%,-3.37%,-1.12%,nan%,-7.33%,-6.85%,-1.04%,-0.40%,-1.00%,-2.19%,-6.88%,-4.13%,-3.71%,nan%,-0.21%,-8.85%,-3.64%,-3.78%,nan%
2024,-7.97%,-2.48%,-2.33%,-11.76%,-1.02%,-0.90%,-1.40%,-0.84%,-2.44%,-8.25%,-12.03%,-16.14%,-1.01%,nan%,-12.14%,-17.43%,-9.19%,-2.57%,-13.68%,-8.24%,-2.30%,-0.77%,-3.24%,nan%,-4.79%,-9.29%,-7.43%,-5.29%,nan%,-18.27%,-3.69%,-0.37%,-12.68%,-19.58%,-21.49%,-1.77%,-2.91%,-12.86%,nan%,-11.34%,-1.07%,-2.75%,-1.61%,-13.08%,-4.48%,-1.03%,-13.26%,-3.86%,nan%,-4.07%,-5.30%,-1.51%,-3.25%,nan%,-5.60%,-12.37%,nan%,nan%,nan%
Avg,-9.88%,-8.44%,-15.52%,-10.25%,-12.48%,-8.31%,-4.16%,-7.17%,-10.44%,-9.73%,-16.68%,-8.29%,-8.55%,-0.44%,-5.33%,-7.80%,-7.97%,-4.85%,-8.85%,-8

C:\Users\nicol\AppData\Local\Temp\ipykernel_30724\3674096426.py:84: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,3.67%,16.39%,13.29%,nan%,1.55%,7.82%,18.91%,10.99%,nan%,4.79%,13.77%,4.89%,5.52%,3.61%,11.45%,22.88%,36.41%,11.27%,nan%,6.02%,74.94%,22.23%,16.17%,nan%
2018,53.48%,28.22%,2.61%,21.73%,0.55%,10.91%,21.01%,5.49%,6.66%,0.59%,2.70%,9.89%,0.71%,nan%,10.70%,33.62%,20.53%,14.87%,21.59%,0.61%,2.03%,0.60%,10.16%,nan%,0.94%,2.71%,10.27%,4.44%,nan%,9.70%,0.50%,14.60%,6.19%,0.73%,1.10%,1.27%,1.58%,10.15%,nan%,0.38%,15.95%,15.84%,0.68%,nan%,1.22%,2.85%,21.42%,0.89%,6.72%,5.97%,0.76%,0.16%,6.87%,nan%,0.28%,2.09%,55.81%,22.20%,15.56%
2019,1.03%,13.91%,1.64%,0.74%,nan%,17.85%,8.38%,24.09%,7.15%,7.42%,6.72%,2.41%,7.18%,nan%,25.74%,6.64%,5.84%,4.29%,8.51%,25.87%,48.75%,3.87%,9.18%,nan%,0.42%,19.93%,20.16%,18.93%,nan%,5.74%,4.42%,4.32%,1.26%,6.11%,7.83%,0.18%,4.77%,3.84%,nan%,7.88%,5.12%,18.46%,0.23%,9.61%,15.62%,4.08%,12.88%,4.84%,nan%,7.47%,0.65%,0.13%,12.58%,nan%,0.84%,0.50%,0.18%,4.50%,2.85%
2020,9.37%,22.48%,2.81%,15.20%,nan%,22.37%,26.20%,10.43%,0.98%,15.67%,4.61%,23.20%,17.21%,20.32%,23.50%,19.54%,11.10%,15.15%,nan%,3.22%,11.85%,4.85%,23.56%,nan%,9.60%,2.33%,2.40%,9.30%,3.48%,9.38%,1.06%,33.30%,33.71%,nan%,9.28%,13.99%,2.86%,10.03%,13.96%,10.65%,7.56%,1.55%,3.60%,nan%,7.40%,5.51%,11.27%,1.30%,nan%,18.05%,5.22%,29.64%,11.24%,10.30%,0.26%,14.46%,12.39%,47.58%,nan%
2021,37.82%,3.49%,16.70%,5.95%,nan%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,nan%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,0.74%,20.57%,nan%,3.72%,1.27%,16.27%,23.19%,nan%,24.83%,10.60%,0.84%,4.32%,24.95%,0.50%,7.99%,0.58%,14.01%,nan%,8.11%,16.31%,13.72%,9.26%,nan%,8.83%,5.54%,3.15%,6.84%,11.27%,6.95%,0.31%,5.76%,1.57%,nan%
2022,1.90%,8.23%,0.32%,7.21%,18.18%,7.48%,11.32%,9.76%,16.32%,8.80%,18.76%,15.31%,8.64%,nan%,0.94%,0.33%,6.42%,4.06%,nan%,4.99%,0.44%,0.00%,2.22%,11.18%,6.30%,1.41%,13.42%,3.43%,nan%,18.82%,21.79%,24.33%,11.71%,nan%,4.28%,19.40%,3.99%,6.42%,15.65%,13.28%,0.96%,5.57%,8.14%,nan%,8.40%,1.51%,5.02%,21.91%,5.63%,2.54%,5.71%,8.18%,9.71%,nan%,2.01%,7.10%,4.64%,1.25%,nan%
2023,7.97%,24.02%,8.16%,2.00%,4.25%,4.23%,15.06%,2.39%,2.22%,1.89%,16.52%,4.30%,4.14%,nan%,8.27%,15.20%,0.30%,5.52%,nan%,7.96%,0.91%,2.64%,6.30%,0.99%,0.44%,0.99%,12.52%,3.18%,nan%,2.00%,8.93%,0.73%,0.11%,0.96%,2.53%,0.86%,1.00%,5.28%,nan%,2.09%,2.27%,2.90%,10.77%,nan%,0.59%,0.19%,7.06%,12.13%,6.55%,12.97%,3.59%,6.04%,7.51%,nan%,9.58%,0.11%,6.68%,8.03%,nan%
2024,6.55%,22.32%,5.72%,0.37%,5.99%,10.91%,15.47%,8.36%,13.18%,14.51%,5.56%,0.01%,6.48%,nan%,0.02%,8.01%,3.89%,6.66%,0.72%,2.71%,7.43%,28.59%,3.93%,nan%,2.82%,0.20%,0.39%,1.95%,nan%,2.52%,11.52%,9.30%,0.76%,3.87%,1.31%,8.79%,7.96%,0.58%,nan%,5.74%,7.29%,13.66%,5.72%,0.22%,3.32%,11.75%,0.82%,8.55%,nan%,32.18%,8.15%,13.70%,11.47%,nan%,10.25%,0.05%,nan%,nan%,nan%
Avg,16.87%,17.52%,5.42%,7.60%,7.24%,15.41%,16.23%,10.57%,6.67%,10.16%,9.64%,8.21%,6.53%,23.76%,10.74%,14.54%,9.48%,11.31%,10.27%,11.49%,11.84%,5.81%,13.45%,11.13%,3.65%,4.68%,8.56%,8.83%,3.48%,7.41%,7.07%,14.69%,10.99%,2.92%,7.31%,7.34%,4.92%,6.74%,18.19%,5.26%,6.87%,9.68%,6.77%,4.92%,6.18%,7.00%,9.64%,8.05%,5.63%,12.43%,6.56%,12.18%,9.69%,10.79%,4.52%,12.45%,15.38%,14.47%,9.21%


C:\Users\nicol\AppData\Local\Temp\ipykernel_30724\3674096426.py:84: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-0.67%,16.39%,-1.82%,nan%,-13.50%,-12.25%,9.50%,7.78%,nan%,2.35%,8.62%,-12.95%,3.82%,-3.10%,3.40%,16.50%,30.32%,-0.55%,nan%,-7.16%,65.59%,-7.01%,11.31%,nan%
2018,53.22%,19.94%,-22.18%,19.35%,-34.22%,-1.53%,12.67%,-7.66%,2.99%,-16.82%,-25.52%,-2.54%,-27.75%,nan%,5.76%,33.34%,16.30%,11.31%,14.97%,-8.21%,-1.77%,-20.35%,8.56%,nan%,-15.42%,-5.24%,-8.60%,-0.48%,nan%,7.66%,-7.56%,1.71%,1.95%,-12.35%,-22.23%,-5.83%,-8.36%,7.75%,nan%,-33.74%,12.26%,11.02%,-4.74%,nan%,-2.96%,-12.93%,5.41%,-1.19%,3.47%,0.19%,-15.82%,-33.72%,-1.75%,nan%,-20.04%,-9.21%,52.37%,6.77%,11.65%
2019,-25.82%,2.89%,-4.69%,-4.18%,nan%,16.58%,6.90%,-1.25%,-1.14%,3.65%,2.44%,-1.86%,3.86%,nan%,23.19%,-4.05%,0.83%,-6.37%,3.29%,15.37%,37.99%,1.96%,1.71%,nan%,-14.09%,15.89%,15.24%,-5.10%,nan%,4.04%,-25.88%,-0.20%,-6.15%,4.83%,-2.42%,-10.20%,-4.00%,-8.47%,nan%,6.13%,4.33%,11.72%,-19.87%,0.50%,6.41%,-3.18%,4.89%,-1.25%,nan%,4.09%,-2.72%,-23.86%,7.67%,nan%,-0.19%,-5.30%,-7.28%,1.70%,0.75%
2020,8.25%,13.80%,0.64%,12.23%,nan%,21.23%,13.27%,6.02%,-20.65%,-8.44%,-37.93%,-1.21%,1.79%,14.63%,11.14%,13.54%,9.64%,6.32%,nan%,-10.64%,10.33%,-3.44%,15.84%,nan%,5.61%,-5.23%,-1.60%,-1.36%,1.19%,6.70%,-1.50%,30.06%,19.74%,nan%,4.75%,11.14%,-9.89%,9.78%,-17.75%,3.85%,1.26%,-3.65%,-1.46%,nan%,6.19%,1.05%,7.34%,-2.26%,nan%,14.53%,-1.37%,24.99%,2.91%,4.38%,-1.81%,8.02%,7.37%,42.79%,nan%
2021,28.20%,-1.74%,12.98%,-5.72%,nan%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,nan%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-11.64%,17.17%,nan%,-7.85%,-11.63%,15.76%,16.71%,nan%,17.86%,9.80%,-2.10%,-0.45%,22.55%,-13.82%,-2.22%,-8.09%,11.71%,nan%,-0.10%,12.67%,6.12%,5.02%,nan%,7.58%,0.30%,-7.96%,0.82%,-2.29%,-1.58%,-4.98%,3.55%,-5.79%,nan%
2022,-17.66%,6.18%,-24.14%,2.47%,17.42%,-6.02%,-8.68%,-0.20%,-2.50%,-1.40%,13.74%,15.18%,6.87%,nan%,-9.05%,-6.71%,-2.24%,-3.29%,nan%,-10.80%,-14.86%,-4.76%,-11.23%,-0.41%,-20.56%,-21.35%,6.14%,-10.31%,nan%,8.76%,14.57%,19.35%,5.03%,nan%,1.32%,13.83%,-16.39%,-11.83%,10.69%,11.83%,-24.44%,-2.99%,-1.38%,nan%,3.64%,-1.30%,4.46%,16.58%,-1.39%,-22.11%,-6.49%,4.52%,7.16%,nan%,-1.28%,-6.33%,3.00%,-1.49%,nan%
2023,7.48%,20.34%,4.83%,1.06%,-0.96%,-7.01%,10.89%,-2.28%,-4.68%,1.01%,12.68%,-0.38%,1.16%,nan%,3.59%,13.97%,-12.11%,0.43%,nan%,0.02%,-3.81%,0.32%,5.75%,-0.98%,-7.26%,-1.82%,10.33%,2.04%,nan%,-3.85%,3.18%,-1.74%,-1.43%,-1.69%,0.57%,-8.45%,-1.60%,-1.36%,nan%,-1.13%,0.31%,-2.57%,9.68%,nan%,-5.82%,-4.60%,6.80%,7.90%,5.38%,8.09%,-1.62%,2.53%,6.33%,nan%,7.27%,-6.63%,3.07%,0.79%,nan%
2024,-2.65%,11.32%,-0.64%,-8.15%,1.46%,9.50%,14.92%,8.03%,12.05%,11.20%,-6.03%,-5.21%,5.51%,nan%,-5.24%,-8.66%,-0.24%,3.68%,-3.89%,-6.60%,4.84%,24.59%,-1.19%,nan%,-1.97%,-2.21%,-5.61%,0.50%,nan%,-14.75%,10.72%,8.96%,-7.52%,-17.77%,-4.97%,2.22%,5.13%,-11.67%,nan%,-5.29%,0.82%,11.44%,2.97%,-8.19%,1.18%,11.26%,-8.70%,-2.00%,nan%,29.52%,-3.37%,9.27%,10.31%,nan%,8.00%,-2.24%,nan%,nan%,nan%
Avg,7.29%,10.39%,-4.74%,2.44%,-4.07%,7.93%,8.81%,1.44%,-5.79%,1.55%,-4.80%,0.06%,-1.99%,18.83%,4.72%,6.48%,2.28%,5.60%,4.79%,1.74%,3.43%,-6.16%,4.75%,4.09%,-8.74%,-4.37%,0.61%,0.35%,1.19%,0.10%,-2.59%,10.56%,4.05%,-6.75%,-0.73%,1.48%,-2.60%,-2.26%,5.16%,-5.71%,-2.49%,3.30%,0.59%,-3.85%,1.36%,1.45%,1.67%,3.33%,1.09%,5.66%,-1.82%,0.76%

In [1]:
import ccxt
import pandas as pd
from IPython.display import display, HTML
from datetime import timedelta

# Function to fetch ETH/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    # Create unique "week number" format: MonthLetter-MonthNumber-WeekOfMonth
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)
    
    return df

# Function to filter data for the 18 months after each halving event
def filter_halving_data(df):
    halving_dates = ['2012-11-28', '2016-07-10', '2020-05-12', '2024-04-20']
    halving_dates = pd.to_datetime(halving_dates)

    # Filter the DataFrame to only include data from 18 months after each halving date
    filtered_df = pd.DataFrame()
    for halving_date in halving_dates:
        start_date = halving_date
        end_date = start_date + timedelta(weeks=18*4)  # 18 months = 18 * 4 weeks
        filtered_df = pd.concat([filtered_df, df[(df.index >= start_date) & (df.index <= end_date)]])

    return filtered_df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to add the halving week number and halving count columns
def add_halving_columns(df):
    halving_dates = pd.to_datetime(['2012-11-28', '2016-07-10', '2020-05-12', '2024-04-20'])
    halving_counts = [1, 2, 3, 4]
    
    # Create new columns to store the week after halving and halving count
    df['week_after_halving'] = None
    df['halving_count'] = None

    # Iterate over each halving event
    for i, halving_date in enumerate(halving_dates):
        # Calculate the weeks since each halving event
        df.loc[df.index >= halving_date, 'week_after_halving'] = ((df.index[df.index >= halving_date] - halving_date).days // 7) + 1
        df.loc[df.index >= halving_date, 'halving_count'] = halving_counts[i]
    
    return df

# Function to create and style the pivot table with positive percentages in green
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('week_after_halving')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='halving_count', columns='week_after_halving', aggfunc='mean')

    # Calculate mean, median, max, and min for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)
    max_value = pivot_table.max(axis=0)
    min_value = pivot_table.min(axis=0)

    # Add the average, median, max, and min as the last rows
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value
    pivot_table.loc['Max'] = max_value
    pivot_table.loc['Min'] = min_value

    # Function to color positive values green and negative values red
    def color_positive_negative(val):
        if isinstance(val, (int, float)):
            if val > 0:
                return 'background-color: lightgreen; color: white;'  # Green for positive values
            elif val < 0:
                return 'background-color: pink; color: white;'  # Red for negative values
        return ''  # No change for non-numeric values

    # Style the table
    styled_table = pivot_table.style \
        .format('{:.2f}%') \
        .applymap(color_positive_negative)  # Apply the coloring function to the table
    
    # Set additional styles
    styled_table.set_table_styles([ 
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%'), ('display', 'block'), ('overflow-x', 'auto'), ('position', 'relative')]},  # Set relative position for the table container
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
        
        # Add a line below the "Avg" row to separate it from the rest of the table
        {'selector': 'tr:nth-last-child(5)', 'props': [('border-bottom', '2px solid black')]},  # Row for "Avg"
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table

# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ETH/USDT'
    ETH_data = fetch_data(symbol)

    # Filter data for 18 months after each halving
    ETH_data = filter_halving_data(ETH_data)

    # Add halving week and halving count columns
    ETH_data = add_halving_columns(ETH_data)

    # Calculate indicators
    ETH_data = calculate_max_drawdown(ETH_data)
    ETH_data = calculate_high_to_open_change(ETH_data)
    ETH_data = calculate_weekly_return(ETH_data)

    # Create and display the pivot tables
    styled_table_high_to_open_change = create_pivot_table(ETH_data, 'high_to_open_change')
    display_table("ETH/USDT Weekly Open to High % Change (Week After Halving)", styled_table_high_to_open_change)
    styled_table_max_drawdown = create_pivot_table(ETH_data, 'max_drawdown')
    display_table("ETH/USDT Weekly Max Drawdown (Week After Halving)", styled_table_max_drawdown)

    styled_table_weekly_return = create_pivot_table(ETH_data, 'weekly_return')
    display_table("ETH/USDT Weekly Returns (Open to Close, Week After Halving)", styled_table_weekly_return)

# Run the main function
main()



C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\2122496671.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,3.67%,16.39%,13.29%,1.55%,7.82%,18.91%,10.99%,4.79%,13.77%,4.89%,5.52%,3.61%,11.45%,22.88%,36.41%
3,4.85%,23.56%,9.60%,2.33%,2.40%,9.30%,3.48%,9.38%,1.06%,33.30%,33.71%,9.28%,13.99%,2.86%,10.03%,13.96%,10.65%,7.56%,1.55%,3.60%,7.40%,5.51%,11.27%,1.30%,18.05%,5.22%,29.64%,11.24%,10.30%,0.26%,14.46%,12.39%,47.58%,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,0.74%,20.57%,3.72%,1.27%,16.27%,23.19%,24.83%,10.60%,0.84%,4.32%,24.95%,0.50%,7.99%,0.58%,14.01%
4,6.66%,0.72%,2.71%,7.43%,28.59%,3.93%,2.82%,0.20%,0.39%,1.95%,2.52%,11.52%,9.30%,0.76%,3.87%,1.31%,8.79%,7.96%,0.58%,5.74%,7.29%,13.66%,5.72%,0.22%,3.32%,11.75%,0.82%,8.55%,32.18%,8.15%,13.70%,11.47%,10.25%,0.05%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,5.76%,12.14%,6.16%,4.88%,15.50%,6.62%,3.15%,4.79%,0.72%,17.63%,18.11%,10.40%,11.64%,1.81%,6.95%,7.64%,9.72%,7.76%,1.07%,4.67%,7.34%,9.59%,8.49%,0.76%,10.68%,8.49%,15.23%,9.90%,21.24%,4.20%,14.08%,11.93%,28.92%,18.93%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,2.21%,18.48%,8.51%,1.41%,12.05%,21.05%,17.91%,7.69%,7.30%,4.60%,15.24%,2.05%,9.72%,11.73%,25.21%
Median,5.76%,12.14%,6.16%,4.88%,15.50%,6.62%,3.15%,4.79%,0.72%,17.63%,18.11%,10.40%,11.64%,1.81%,6.95%,7.64%,9.72%,7.76%,1.07%,4.67%,7.34%,9.59%,8.49%,0.76%,10.68%,8.49%,15.23%,9.90%,21.24%,4.20%,14.08%,11.93%,28.92%,18.93%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,2.21%,18.48%,8.51%,1.41%,12.05%,21.05%,17.91%,7.69%,7.30%,4.60%,15.24%,2.05%,9.72%,11.73%,25.21%
Max,6.66%,23.56%,9.60%,7.43%,28.59%,9.30%,3.48%,9.38%,1.06%,33.30%,33.71%,11.52%,13.99%,2.86%,10.03%,13.96%,10.65%,7.96%,1.55%,5.74%,7.40%,13.66%,11.27%,1.30%,18.05%,11.75%,29.64%,11.24%,32.18%,8.15%,14.46%,12.39%,47.58%,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,3.67%,20.57%,13.29%,1.55%,16.27%,23.19%,24.83%,10.60%,13.77%,4.89%,24.95%,3.61%,11.45%,22.88%,36.41%
Min,4.85%,0.72%,2.71%,2.33%,2.40%,3.93%,2.82%,0.20%,0.39%,1.95%,2.52%,9.28%,9.30%,0.76%,3.87%,1.31%,8.79%,7.56%,0.58%,3.60%,7.29%,5.51%,5.72%,0.22%,3.32%,5.22%,0.82%,8.55%,10.30%,0.26%,13.70%,11.47%,10.25%,0.05%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,0.74%,16.39%,3.72%,1.27%,7.82%,18.91%,10.99%,4.79%,0.84%,4.32%,5.52%,0.50%,7.99%,0.58%,14.01%


C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\2122496671.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-7.68%,-51.79%,-8.05%,-30.58%,-34.58%,-2.11%,-2.84%,-6.83%,-7.87%,-15.65%,-7.35%,-9.92%,-4.80%,-0.10%,-1.44%
3,-7.40%,-1.50%,-2.83%,-7.51%,-5.81%,-5.28%,-1.61%,-0.27%,-5.25%,-2.12%,-1.66%,-2.58%,-6.23%,-12.38%,-5.30%,-28.10%,-8.46%,-3.12%,-15.56%,-6.53%,-5.72%,-3.47%,-3.40%,-8.02%,-6.67%,-4.67%,-0.58%,-14.11%,-2.83%,-11.83%,-2.57%,-13.61%,-0.59%,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-24.20%,-1.10%,-11.89%,-13.83%,-9.81%,-1.80%,-4.43%,-3.94%,-10.80%,-5.70%,-2.50%,-23.93%,-8.69%,-20.33%,-9.07%
4,-2.57%,-13.68%,-8.24%,-2.30%,-0.77%,-3.24%,-4.79%,-9.29%,-7.43%,-5.29%,-18.27%,-3.69%,-0.37%,-12.68%,-19.58%,-21.49%,-1.77%,-2.91%,-12.86%,-11.34%,-1.07%,-2.75%,-1.61%,-13.08%,-4.48%,-1.03%,-13.26%,-3.86%,-4.07%,-5.30%,-1.51%,-3.25%,-5.60%,-12.37%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,-4.98%,-7.59%,-5.54%,-4.90%,-3.29%,-4.26%,-3.20%,-4.78%,-6.34%,-3.70%,-9.96%,-3.13%,-3.30%,-12.53%,-12.44%,-24.79%,-5.12%,-3.01%,-14.21%,-8.94%,-3.39%,-3.11%,-2.50%,-10.55%,-5.57%,-2.85%,-6.92%,-8.99%,-3.45%,-8.56%,-2.04%,-8.43%,-3.10%,-10.70%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-15.94%,-26.44%,-9.97%,-22.20%,-22.20%,-1.95%,-3.63%,-5.38%,-9.34%,-10.68%,-4.92%,-16.93%,-6.75%,-10.21%,-5.26%
Median,-4.98%,-7.59%,-5.54%,-4.90%,-3.29%,-4.26%,-3.20%,-4.78%,-6.34%,-3.70%,-9.96%,-3.13%,-3.30%,-12.53%,-12.44%,-24.79%,-5.12%,-3.01%,-14.21%,-8.94%,-3.39%,-3.11%,-2.50%,-10.55%,-5.57%,-2.85%,-6.92%,-8.99%,-3.45%,-8.56%,-2.04%,-8.43%,-3.10%,-10.70%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-15.94%,-26.44%,-9.97%,-22.20%,-22.20%,-1.95%,-3.63%,-5.38%,-9.34%,-10.68%,-4.92%,-16.93%,-6.75%,-10.21%,-5.26%
Max,-2.57%,-1.50%,-2.83%,-2.30%,-0.77%,-3.24%,-1.61%,-0.27%,-5.25%,-2.12%,-1.66%,-2.58%,-0.37%,-12.38%,-5.30%,-21.49%,-1.77%,-2.91%,-12.86%,-6.53%,-1.07%,-2.75%,-1.61%,-8.02%,-4.48%,-1.03%,-0.58%,-3.86%,-2.83%,-5.30%,-1.51%,-3.25%,-0.59%,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-7.68%,-1.10%,-8.05%,-13.83%,-9.81%,-1.80%,-2.84%,-3.94%,-7.87%,-5.70%,-2.50%,-9.92%,-4.80%,-0.10%,-1.44%
Min,-7.40%,-13.68%,-8.24%,-7.51%,-5.81%,-5.28%,-4.79%,-9.29%,-7.43%,-5.29%,-18.27%,-3.69%,-6.23%,-12.68%,-19.58%,-28.10%,-8.46%,-3.12%,-15.56%,-11.34%,-5.72%,-3.47%,-3.40%,-13.08%,-6.67%,-4.67%,-13.26%,-14.11%,-4.07%,-11.83%,-2.57%,-13.61%,-5.60%,-12.37%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-24.20%,-51.79%,-11.89%,-30.58%,-34.58%,-2.11%,-4.43%,-6.83%,-10.80%,-15.65%,-7.35%,-23.93%,-8.69%,-20.33%,-9.07%


C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\2122496671.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-0.67%,16.39%,-1.82%,-13.50%,-12.25%,9.50%,7.78%,2.35%,8.62%,-12.95%,3.82%,-3.10%,3.40%,16.50%,30.32%
3,-3.44%,15.84%,5.61%,-5.23%,-1.60%,-1.36%,1.19%,6.70%,-1.50%,30.06%,19.74%,4.75%,11.14%,-9.89%,9.78%,-17.75%,3.85%,1.26%,-3.65%,-1.46%,6.19%,1.05%,7.34%,-2.26%,14.53%,-1.37%,24.99%,2.91%,4.38%,-1.81%,8.02%,7.37%,42.79%,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-11.64%,17.17%,-7.85%,-11.63%,15.76%,16.71%,17.86%,9.80%,-2.10%,-0.45%,22.55%,-13.82%,-2.22%,-8.09%,11.71%
4,3.68%,-3.89%,-6.60%,4.84%,24.59%,-1.19%,-1.97%,-2.21%,-5.61%,0.50%,-14.75%,10.72%,8.96%,-7.52%,-17.77%,-4.97%,2.22%,5.13%,-11.67%,-5.29%,0.82%,11.44%,2.97%,-8.19%,1.18%,11.26%,-8.70%,-2.00%,29.52%,-3.37%,9.27%,10.31%,8.00%,-2.20%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,0.12%,5.98%,-0.49%,-0.19%,11.50%,-1.28%,-0.39%,2.24%,-3.56%,15.28%,2.50%,7.73%,10.05%,-8.70%,-4.00%,-11.36%,3.03%,3.19%,-7.66%,-3.38%,3.51%,6.25%,5.15%,-5.22%,7.85%,4.95%,8.14%,0.45%,16.95%,-2.59%,8.64%,8.84%,25.40%,13.00%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-6.16%,16.78%,-4.83%,-12.56%,1.75%,13.10%,12.82%,6.07%,3.26%,-6.70%,13.18%,-8.46%,0.59%,4.20%,21.02%
Median,0.12%,5.98%,-0.49%,-0.19%,11.50%,-1.28%,-0.39%,2.24%,-3.56%,15.28%,2.50%,7.73%,10.05%,-8.70%,-4.00%,-11.36%,3.03%,3.19%,-7.66%,-3.38%,3.51%,6.25%,5.15%,-5.22%,7.85%,4.95%,8.14%,0.45%,16.95%,-2.59%,8.64%,8.84%,25.40%,13.00%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-6.16%,16.78%,-4.83%,-12.56%,1.75%,13.10%,12.82%,6.07%,3.26%,-6.70%,13.18%,-8.46%,0.59%,4.20%,21.02%
Max,3.68%,15.84%,5.61%,4.84%,24.59%,-1.19%,1.19%,6.70%,-1.50%,30.06%,19.74%,10.72%,11.14%,-7.52%,9.78%,-4.97%,3.85%,5.13%,-3.65%,-1.46%,6.19%,11.44%,7.34%,-2.26%,14.53%,11.26%,24.99%,2.91%,29.52%,-1.81%,9.27%,10.31%,42.79%,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-0.67%,17.17%,-1.82%,-11.63%,15.76%,16.71%,17.86%,9.80%,8.62%,-0.45%,22.55%,-3.10%,3.40%,16.50%,30.32%
Min,-3.44%,-3.89%,-6.60%,-5.23%,-1.60%,-1.36%,-1.97%,-2.21%,-5.61%,0.50%,-14.75%,4.75%,8.96%,-9.89%,-17.77%,-17.75%,2.22%,1.26%,-11.67%,-5.29%,0.82%,1.05%,2.97%,-8.19%,1.18%,-1.37%,-8.70%,-2.00%,4.38%,-3.37%,8.02%,7.37%,8.00%,-2.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-11.64%,16.39%,-7.85%,-13.50%,-12.25%,9.50%,7.78%,2.35%,-2.10%,-12.95%,3.82%,-13.82%,-2.22%,-8.09%,11.71%


In [2]:
import ccxt
import pandas as pd
from IPython.display import display, HTML

# Function to fetch ETH/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    return df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to create a heatmap-style pivot table
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('unique_week_number')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='year', columns='unique_week_number', aggfunc='mean')

    # Calculate mean and median for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)

    # Add the average and median as the last row
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value

    # Apply color gradients for heatmap (Green for positive, Red for negative)
    def color_map(val):
        color = 'white'  # Default color for no value
        if isinstance(val, (int, float)):  # Check if value is numeric
            if val > 0:
                color = f'rgb({255 - int(val*2.5)}, 255, {255 - int(val*2.5)})'  # Green shades for positive
            elif val < 0:
                color = f'rgb(255, {255 + int(val*2.5)}, {255 - int(abs(val)*2.5)})'  # Red shades for negative
        return f'background-color: {color};'

    # Apply heatmap coloring
    styled_table = pivot_table.style \
        .applymap(color_map)  # Apply the heatmap color function to each cell

    # Additional table styling
    styled_table = styled_table.set_table_styles([
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table

# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ETH/USDT'
    eth_data = fetch_data(symbol)

    # Calculate indicators
    eth_data = calculate_max_drawdown(eth_data)
    eth_data = calculate_high_to_open_change(eth_data)
    eth_data = calculate_weekly_return(eth_data)

    # Create and display the pivot tables with heatmap style
    styled_table_max_drawdown = create_pivot_table(eth_data, 'max_drawdown')
    display_table("ETH/USDT Weekly Max Drawdown Heatmap (Month-Letter-WeekOfMonth)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(eth_data, 'high_to_open_change')
    display_table("ETH/USDT Weekly Open to High % Change Heatmap (Month-Letter-WeekOfMonth)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(eth_data, 'weekly_return')
    display_table("ETH/USDT Weekly Returns Heatmap (Open to Close, Month-Letter-WeekOfMonth)", styled_table_weekly_return)

# Run the main function
main()


C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\2132610971.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-7.681068,-51.785356,-8.052053,nan,-30.575614,-34.584852,-2.108492,-2.836879,nan,-6.830295,-7.868094,-15.650370,-7.345633,-9.924590,-4.803686,-0.098684,-1.444460,-16.817907,nan,-18.529220,0.000000,-28.635426,-9.009706,nan
2018,-2.211430,-18.973546,-43.990682,-13.441373,-37.404550,-30.611003,-0.138514,-13.722283,-1.005395,-26.149017,-37.586685,-5.709075,-31.882813,nan,-4.236885,-3.321678,-6.849854,-4.324359,-9.158283,-20.282009,-10.023591,-23.201519,-11.334770,nan,-19.128232,-14.406069,-15.612994,-10.964362,nan,-1.764940,-14.306854,-2.176571,-2.264629,-14.963551,-25.520464,-21.373758,-13.297695,-0.941022,nan,-37.389598,-14.689237,-12.640756,-16.737548,nan,-7.093203,-16.463361,-1.137980,-2.564350,-5.844188,-2.126055,-19.571966,-42.498883,-13.729456,nan,-29.632170,-12.298949,-0.330657,-13.802992,-6.610551
2019,-27.305218,-0.201454,-5.546104,-9.933952,nan,-5.702040,-5.546675,-1.547432,-3.981832,-6.125574,-5.060205,-5.379678,-3.320599,nan,-1.301181,-9.283378,-6.883560,-10.819147,-3.712150,-3.373198,-1.462222,-10.899487,-9.158313,nan,-15.736229,-1.727123,-2.345647,-9.211465,nan,-6.910444,-26.880415,-15.051145,-12.084793,-3.393515,-8.652329,-17.730234,-7.611157,-12.292270,nan,-3.356176,-4.514847,-0.396720,-28.963068,-2.510930,-0.823142,-6.807382,-12.404384,-3.709748,nan,-0.644522,-5.974809,-24.926559,-6.100436,nan,-5.022558,-7.444829,-18.391127,-7.727823,-6.311402
2020,-0.376745,-2.927129,-6.739013,-1.602049,nan,-2.010823,-5.320964,-6.524006,-23.497268,-10.242872,-56.877100,-18.349217,-2.297441,-0.329397,-0.063074,-6.451816,-7.399178,-4.103551,nan,-14.423902,-6.193370,-7.395421,-1.500750,nan,-2.832959,-7.511142,-5.812549,-5.275622,-1.614122,-0.268085,-5.247982,-2.120274,-1.659057,nan,-2.577527,-6.225970,-12.382095,-5.303030,-28.097170,-8.459685,-3.119286,-15.564360,-6.527211,nan,-5.721746,-3.471684,-3.398392,-8.022668,nan,-6.667843,-4.666623,-0.584730,-14.111744,-2.831760,-11.828654,-2.574432,-13.606018,-0.592614,nan
2021,-9.028651,-27.047455,-15.431237,-13.302064,nan,-3.235171,-2.951981,-8.019866,-33.118529,-0.617480,-3.562202,-7.381635,-13.812576,-0.551815,-7.018837,-9.510928,-8.080424,-0.724001,nan,-0.000339,-14.803705,-51.735575,-0.796039,-4.794159,-16.803593,-18.676300,-24.203807,-1.097227,nan,-11.890523,-13.832491,-9.814183,-1.795581,nan,-4.428879,-3.938124,-10.804723,-5.704138,-2.496378,-23.929443,-8.689305,-20.326146,-9.065948,nan,-4.322460,-1.265882,-4.445022,-4.707116,nan,-3.292843,-3.266278,-14.483087,-8.139071,-18.419846,-8.774580,-11.774092,-4.361210,-11.777083,nan
2022,-21.632346,-7.079382,-31.279486,-14.966522,-4.827668,-7.145663,-10.279541,-12.280367,-3.125585,-4.166503,-0.675369,-1.103659,-2.578247,nan,-10.754138,-7.899995,-3.508954,-7.248570,nan,-12.064882,-28.563208,-11.285654,-16.649944,-4.225756,-21.216685,-38.560397,-7.521289,-16.679885,nan,-2.689293,-13.869750,-0.192731,-15.117356,nan,-7.081734,-2.553817,-21.270384,-11.997182,-0.328016,-5.612016,-24.945644,-8.580678,-3.199394,nan,-1.071496,-10.061748,-4.082055,-2.933609,-5.541199,-31.545991,-7.273844,-5.884207,-3.589976,nan,-4.798343,-8.525732,-2.751340,-3.074221,nan
2023,-0.610665,-0.365074,-3.293355,-6.724099,-6.671612,-8.533404,-3.491504,-7.287437,-5.908479,-12.538908,-0.970051,-3.961992,-4.946783,nan,-1.768728,-0.672975,-13.860648,-4.135853,nan,-3.420175,-

C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\2132610971.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3.669512,16.392511,13.294648,nan,1.550296,7.819154,18.906535,10.992908,nan,4.790419,13.765950,4.888889,5.521138,3.606557,11.453640,22.881579,36.412783,11.269553,nan,6.017814,74.938575,22.226619,16.173794,nan
2018,53.476760,28.217685,2.606844,21.734986,0.553532,10.905550,21.014618,5.488474,6.658665,0.593116,2.704577,9.886674,0.709991,nan,10.697871,33.616384,20.530831,14.874249,21.587899,0.609342,2.032699,0.600424,10.160999,nan,0.937611,2.714242,10.268531,4.442096,nan,9.696112,0.499435,14.602370,6.186198,0.726844,1.103310,1.267455,1.583695,10.150636,nan,0.375622,15.953704,15.841809,0.678826,nan,1.218675,2.846283,21.418411,0.886918,6.717401,5.972753,0.761994,0.161975,6.868942,nan,0.282946,2.090929,55.810109,22.196388,15.564908
2019,1.033206,13.909083,1.641715,0.740807,nan,17.851486,8.383951,24.093593,7.153671,7.419602,6.722317,2.408596,7.177490,nan,25.740754,6.644099,5.836971,4.294442,8.513873,25.869365,48.753124,3.867062,9.181010,nan,0.420277,19.929010,20.158369,18.928734,nan,5.735942,4.420452,4.317407,1.263914,6.111164,7.827224,0.184834,4.770482,3.838319,nan,7.877936,5.116459,18.455435,0.227273,9.612431,15.622060,4.083324,12.878182,4.841166,nan,7.469840,0.650931,0.130563,12.579470,nan,0.835987,0.498538,0.182507,4.503482,2.850551
2020,9.366920,22.475437,2.805924,15.198618,nan,22.373727,26.202249,10.425277,0.983607,15.665029,4.608133,23.203037,17.210367,20.318149,23.498493,19.538345,11.098767,15.152743,nan,3.216430,11.848417,4.854603,23.561781,nan,9.604422,2.334710,2.399241,9.301408,3.477256,9.378571,1.058659,33.301271,33.705228,nan,9.276413,13.987287,2.857406,10.025082,13.964657,10.650418,7.562154,1.552393,3.599195,nan,7.396265,5.513537,11.268909,1.296045,nan,18.049813,5.221334,29.640458,11.244040,10.303304,0.259313,14.461502,12.385206,47.579915,nan
2021,37.819550,3.488965,16.697769,5.948182,nan,34.137681,16.150572,13.461442,0.142226,22.238435,12.617726,2.362426,1.361574,27.195998,5.988341,18.446886,18.286404,28.639396,nan,35.069660,11.436690,0.146574,38.794160,21.214090,5.001254,5.207366,0.743039,20.565311,nan,3.715950,1.274912,16.274152,23.188664,nan,24.828409,10.596180,0.836017,4.320259,24.954785,0.499205,7.989308,0.578326,14.010773,nan,8.112695,16.307678,13.724981,9.262847,nan,8.825464,5.537236,3.145142,6.838510,11.269055,6.947794,0.308717,5.755656,1.572263,nan
2022,1.897020,8.231677,0.319102,7.207562,18.176402,7.484779,11.321287,9.763958,16.322288,8.801395,18.764062,15.306547,8.638357,nan,0.939834,0.330623,6.423210,4.058555,nan,4.992902,0.439336,0.000000,2.217611,11.182484,6.299343,1.405035,13.421885,3.434659,nan,18.822259,21.785907,24.329735,11.709958,nan,4.279193,19.397718,3.986421,6.419138,15.646640,13.278752,0.955767,5.574331,8.139840,nan,8.401999,1.510811,5.022359,21.908239,5.630482,2.537223,5.706099,8.181508,9.707006,nan,2.009536,7.103721,4.643044,1.248236,nan
2023,7.970308,24.015037,8.163502,2.001315,4.252978,4.229563,15.061195,2.390251,2.223143,1.893172,16.517436,4.295574,4.137822,nan,8.273568,15.204071,0.299246,5.516082,nan,7.963296,0.912099,2.640062,6.301699,0.994425,0.438624,0.990432,12.520410,3.180956,nan,1.995881,8.927373,0.731484,0.112244,0.964126,2.525843,0.856725,1.001644,5.276996,nan,2.088224,2.266573,2.898033,10.773007,nan,0.588303,0.192914,7.062615,12.132669,6.547122,12.966047,3.585385,6.042844,7.512340,

C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\2132610971.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-0.674127,16.392511,-1.821263,nan,-13.495830,-12.251712,9.501806,7.783688,nan,2.345858,8.623405,-12.951111,3.819184,-3.104918,3.404587,16.496711,30.322399,-0.548786,nan,-7.158375,65.587926,-7.012238,11.310282,nan
2018,53.216191,19.936960,-22.180432,19.347950,-34.218346,-1.534810,12.666650,-7.663008,2.994795,-16.819561,-25.520111,-2.540055,-27.745014,nan,5.763643,33.336663,16.303289,11.307136,14.974307,-8.211003,-1.772096,-20.352994,8.558014,nan,-15.419746,-5.241690,-8.597003,-0.482071,nan,7.659132,-7.557291,1.707463,1.952041,-12.352058,-22.232551,-5.830922,-8.356164,7.747018,nan,-33.738283,12.257176,11.015256,-4.743600,nan,-2.960865,-12.927605,5.410486,-1.190591,3.473340,0.193278,-15.818438,-33.718722,-1.753055,nan,-20.037726,-9.210808,52.373642,6.773619,11.646470
2019,-25.817142,2.890427,-4.686968,-4.177080,nan,16.575049,6.897379,-1.248411,-1.135503,3.652374,2.437763,-1.860532,3.864237,nan,23.194965,-4.050720,0.825309,-6.370485,3.287723,15.369519,37.985856,1.958592,1.713637,nan,-14.092312,15.890399,15.235498,-5.096946,nan,4.039059,-25.882778,-0.199265,-6.151049,4.825598,-2.421209,-10.202856,-4.003705,-8.470033,nan,6.126633,4.332708,11.716477,-19.867424,0.496278,6.414628,-3.182672,4.892111,-1.251088,nan,4.092987,-2.720152,-23.860298,7.671977,nan,-0.185775,-5.304440,-7.279236,1.695428,0.751712
2020,8.251459,13.803221,0.641525,12.226788,nan,21.227716,13.267405,6.021863,-20.652095,-8.438547,-37.933109,-1.211436,1.790532,14.630031,11.143037,13.540616,9.643373,6.317443,nan,-10.635662,10.329389,-3.439281,15.842921,nan,5.614096,-5.233675,-1.596617,-1.359470,1.187247,6.702118,-1.503543,30.060221,19.741496,nan,4.749631,11.144717,-9.893683,9.781941,-17.750734,3.845827,1.258085,-3.651897,-1.457017,nan,6.192882,1.052997,7.336295,-2.261919,nan,14.525562,-1.366968,24.987167,2.905202,4.376041,-1.810203,8.017806,7.367184,42.793753,nan
2021,28.203163,-1.737307,12.982556,-5.720484,nan,22.794773,11.726012,7.416543,-26.601984,21.674526,7.045631,-3.523619,-5.395827,23.028640,3.645035,3.914624,3.794870,27.112099,nan,33.045471,-8.711885,-41.471211,13.800718,13.668257,-7.470353,-10.594860,-11.643020,17.173241,nan,-7.846075,-11.627907,15.760844,16.707766,nan,17.857562,9.802978,-2.096845,-0.448734,22.547751,-13.823577,-2.223071,-8.087557,11.713888,nan,-0.102700,12.671415,6.117234,5.018703,nan,7.582776,0.301566,-7.958284,0.823532,-2.289529,-1.583247,-4.984504,3.552880,-5.790475,nan
2022,-17.662502,6.183752,-24.138230,2.468688,17.424193,-6.018658,-8.677738,-0.198323,-2.503067,-1.397327,13.742716,15.176540,6.865413,nan,-9.053636,-6.711749,-2.244593,-3.292719,nan,-10.800726,-14.859647,-4.759263,-11.234883,-0.408571,-20.561172,-21.348025,6.137187,-10.313160,nan,8.759518,14.574150,19.351586,5.033486,nan,1.315162,13.828373,-16.389538,-11.826615,10.689955,11.829366,-24.438317,-2.988363,-1.382645,nan,3.635096,-1.298436,4.459555,16.584933,-1.393316,-22.111271,-6.491199,4.523687,7.164037,nan,-1.281069,-6.329375,2.996467,-1.491974,nan
2023,7.483775,20.336395,4.825703,1.062411,-0.955172,-7.009165,10.887762,-2.284877,-4.678409,1.007306,12.677732,-0.381317,1.162987,nan,3.589281,13.973780,-12.114261,0.433940,nan,0.017112,-3.806652,0.317363,5.747101,-0.976088,-7.261376,-1.815412,10.334408,2.037076,nan,-3.854988,3.183363,-1.737144,-1.426877,-1.692457,0.570951,-8.447192

In [3]:
import ccxt
import pandas as pd
from IPython.display import display, HTML
from datetime import timedelta

# Function to fetch ETH/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    # Create unique "week number" format: MonthLetter-MonthNumber-WeekOfMonth
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)
    return df

# Function to filter data for the 18 months after each halving event
def filter_halving_data(df):
    halving_dates = ['2012-11-28', '2016-07-10', '2020-05-12', '2024-04-20']
    halving_dates = pd.to_datetime(halving_dates)

    # Filter the DataFrame to only include data from 18 months after each halving date
    filtered_df = pd.DataFrame()
    for halving_date in halving_dates:
        start_date = halving_date
        end_date = start_date + timedelta(weeks=18*4)  # 18 months = 18 * 4 weeks
        filtered_df = pd.concat([filtered_df, df[(df.index >= start_date) & (df.index <= end_date)]])

    return filtered_df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to create and style the pivot table with positive percentages in green
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('unique_week_number')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='year', columns='unique_week_number', aggfunc='mean')

    # Calculate mean, median, max, and min for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)
    max_value = pivot_table.max(axis=0)
    min_value = pivot_table.min(axis=0)

    # Add the average, median, max, and min as the last rows
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value
    pivot_table.loc['Max'] = max_value
    pivot_table.loc['Min'] = min_value

    # Function to color positive values green and negative values red
    def color_positive_negative(val):
        if isinstance(val, (int, float)):
            if val > 0:
                return 'background-color: lightgreen; color: white;'  # Green for positive values
            elif val < 0:
                return 'background-color: pink; color: white;'  # Red for negative values
        return ''  # No change for non-numeric values

    # Style the table
    styled_table = pivot_table.style \
        .format('{:.2f}%') \
        .applymap(color_positive_negative)  # Apply the coloring function to the table
    
    # Set additional styles
    styled_table.set_table_styles([ 
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%'), ('display', 'block'), ('overflow-x', 'auto'), ('position', 'relative')]},  # Set relative position for the table container
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
        
        # Add a line below the "Avg" row to separate it from the rest of the table
        {'selector': 'tr:nth-last-child(5)', 'props': [('border-bottom', '2px solid black')]},  # Row for "Avg"
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table

# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ETH/USDT'
    ETH_data = fetch_data(symbol)

    # Filter data for 18 months after each halving
    ETH_data = filter_halving_data(ETH_data)

    # Calculate indicators
    ETH_data = calculate_max_drawdown(ETH_data)
    ETH_data = calculate_high_to_open_change(ETH_data)
    ETH_data = calculate_weekly_return(ETH_data)

    # Create and display the pivot tables
    styled_table_max_drawdown = create_pivot_table(ETH_data, 'max_drawdown')
    display_table("ETH/USDT Weekly Max Drawdown (Month-Letter-WeekOfMonth)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(ETH_data, 'high_to_open_change')
    display_table("ETH/USDT Weekly Open to High % Change (Month-Letter-WeekOfMonth)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(ETH_data, 'weekly_return')
    display_table("ETH/USDT Weekly Returns (Open to Close, Month-Letter-WeekOfMonth)", styled_table_weekly_return)

# Run the main function
main()


# 第一次减半	2012-11-28	50→25
# 第二次减半	2016-07-10	25→12.5
# 第三次减半	2020-05-12	12.5→6.25
# 第四次减半	2024-04-20	6.25→3.125




# 然後help me build a new col in the df fetched, and name it "減半後的第幾周“（make it english name）

#  find該周是減半後的第幾周然後填入col of the df
# 再create一個col“第幾次減半”

# 然後幫我改變三幅table的code，我要“減半後的第幾周”作為x axis ，第幾次減半作為y axis

# 三個table：
# 應該有大约（18x4）個col （18个月）
# 和4個row（四個減半）



C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\1418307944.py:90: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-7.68%,-51.79%,-8.05%,nan%,-30.58%,-34.58%,-2.11%,-2.84%,nan%,-6.83%,-7.87%,-15.65%,-7.35%,-9.92%,-4.80%,-0.10%,-1.44%,nan%,nan%,nan%,nan%,nan%,nan%
2020,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-7.40%,-1.50%,nan%,-2.83%,-7.51%,-5.81%,-5.28%,-1.61%,-0.27%,-5.25%,-2.12%,-1.66%,nan%,-2.58%,-6.23%,-12.38%,-5.30%,-28.10%,-8.46%,-3.12%,-15.56%,-6.53%,nan%,-5.72%,-3.47%,-3.40%,-8.02%,nan%,-6.67%,-4.67%,-0.58%,-14.11%,-2.83%,-11.83%,-2.57%,-13.61%,-0.59%
2021,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,nan%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-24.20%,-1.10%,nan%,-11.89%,-13.83%,-9.81%,-1.80%,nan%,-4.43%,-3.94%,-10.80%,-5.70%,-2.50%,-23.93%,-8.69%,-20.33%,-9.07%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-2.57%,-13.68%,-8.24%,-2.30%,-0.77%,-3.24%,nan%,-4.79%,-9.29%,-7.43%,-5.29%,nan%,-18.27%,-3.69%,-0.37%,-12.68%,-19.58%,-21.49%,-1.77%,-2.91%,-12.86%,nan%,-11.34%,-1.07%,-2.75%,-1.61%,-13.08%,-4.48%,-1.03%,-13.26%,-3.86%,nan%,-4.07%,-5.30%,-1.51%,-3.25%,nan%,-5.60%,-12.37%,nan%,nan%
Avg,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-1.65%,-13.68%,-4.12%,-8.55%,-19.97%,-1.84%,-4.79%,-8.14%,-11.82%,-12.48%,-3.89%,-1.61%,-10.14%,-7.59%,-4.10%,-5.38%,-19.58%,-9.50%,-4.90%,-19.47%,-7.98%,-15.30%,-18.58%,-11.86%,-10.19%,-5.01%,-13.08%,-5.68%,-4.12%,-10.77%,-6.41%,-9.92%,-5.18%,-3.36%,-1.18%,-8.68%,-2.83%,-8.71%,-7.47%,-13.61%,-0.59%
Median,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-1.65%,-13.68%,-4.12%,-8.55%,-7.40%,-1.50%,-4.79%,-4.79%,-9.29%,-7.43%,-5.28%,-1.61%,-11.89%,-5.25%,-2.12%,-1.80%,-19.58%,-4.43%,-5.08%,-11.59%,-6.88%,-15.30%,-17.64%,-5.90%,-9.16%,-4.68%,-13.08%,-5.72%,-3.47%,-13.26%,-7.35%,-9.92%,-4.80%,-4.67%,-1.44%,-8.68%,-2.83%,-8.71%,-7.47%,-13.61%,-0.59%
Max,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-13.68%,-0.00%,-2.30%,-0.77%,-0.80%,-4.79%,-2.83%,-7.51%,-5.81%,-1.10%,-1.61%,-0.27%,-3.69%,-0.37%,-1.66%,-19.58%,-2.58%,-1.77%,-2.91%,-5.30%,-2.50%,-8.46%,-1.07%,-2.11%,-1.61%,-13.08%,-4.48%,-1.03%,-3.40%,-3.86%,-9.92%,-4.07%,-0.10%,-0.58%,-3.25%,-2.83%,-5.60%,-2.57%,-13.61%,-0.59%
Min,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-2.57%,-13.68%,-8.24%,-14.80%,-51.74%,-3.24%,-4.79%,-16.80%,-18.68%,-24.20%,-5.29%,-1.61%,-18.27%,-13.83%,-9.81%,-12.68%,-19.58%,-21.49%,-7.68%,-51.79%,-12.86%,-28.10%,-30.58%,-34.58%,-20.33%,-9.07%,-13.08%,-6.83%,-7.87%,-15.65%,-8.02%,-9.92%,-6.67%,-5.30%,-1.51%,-14.11%,-2.83%,-11.83%,-12.37%,-13.61%,-0.59%


C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\1418307944.py:90: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,3.67%,16.39%,13.29%,nan%,1.55%,7.82%,18.91%,10.99%,nan%,4.79%,13.77%,4.89%,5.52%,3.61%,11.45%,22.88%,36.41%,nan%,nan%,nan%,nan%,nan%,nan%
2020,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,4.85%,23.56%,nan%,9.60%,2.33%,2.40%,9.30%,3.48%,9.38%,1.06%,33.30%,33.71%,nan%,9.28%,13.99%,2.86%,10.03%,13.96%,10.65%,7.56%,1.55%,3.60%,nan%,7.40%,5.51%,11.27%,1.30%,nan%,18.05%,5.22%,29.64%,11.24%,10.30%,0.26%,14.46%,12.39%,47.58%
2021,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,nan%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,0.74%,20.57%,nan%,3.72%,1.27%,16.27%,23.19%,nan%,24.83%,10.60%,0.84%,4.32%,24.95%,0.50%,7.99%,0.58%,14.01%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,6.66%,0.72%,2.71%,7.43%,28.59%,3.93%,nan%,2.82%,0.20%,0.39%,1.95%,nan%,2.52%,11.52%,9.30%,0.76%,3.87%,1.31%,8.79%,7.96%,0.58%,nan%,5.74%,7.29%,13.66%,5.72%,0.22%,3.32%,11.75%,0.82%,8.55%,nan%,32.18%,8.15%,13.70%,11.47%,nan%,10.25%,0.05%,nan%,nan%
Avg,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,17.65%,0.72%,18.89%,9.43%,11.20%,22.10%,21.21%,5.81%,2.58%,1.18%,10.61%,3.48%,5.21%,4.62%,19.63%,19.22%,3.87%,11.80%,9.26%,7.01%,7.05%,19.46%,4.61%,7.67%,8.68%,8.58%,0.22%,5.17%,10.34%,5.66%,5.12%,3.61%,20.56%,12.08%,26.59%,11.36%,10.30%,5.26%,7.26%,12.39%,47.58%
Median,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,17.65%,0.72%,18.89%,9.43%,4.85%,23.56%,21.21%,5.00%,2.33%,0.74%,9.30%,3.48%,3.72%,1.27%,16.27%,23.19%,3.87%,9.28%,9.69%,5.41%,7.17%,19.46%,3.64%,7.69%,7.61%,8.36%,0.22%,4.79%,11.75%,4.89%,5.52%,3.61%,18.05%,8.15%,29.64%,11.36%,10.30%,5.26%,7.26%,12.39%,47.58%
Max,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,0.72%,35.07%,11.44%,28.59%,38.79%,21.21%,9.60%,5.21%,2.40%,20.57%,3.48%,9.38%,11.52%,33.30%,33.71%,3.87%,24.83%,13.99%,16.39%,13.29%,24.95%,10.65%,7.99%,18.91%,14.01%,0.22%,7.40%,13.77%,11.27%,8.55%,3.61%,32.18%,22.88%,36.41%,11.47%,10.30%,10.25%,14.46%,12.39%,47.58%
Min,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,6.66%,0.72%,2.71%,7.43%,0.15%,3.93%,21.21%,2.82%,0.20%,0.39%,1.95%,3.48%,2.52%,1.06%,9.30%,0.76%,3.87%,1.31%,3.67%,0.84%,0.58%,13.96%,0.50%,7.29%,0.58%,3.60%,0.22%,3.32%,5.51%,0.82%,1.30%,3.61%,11.45%,5.22%,13.70%,11.24%,10.30%,0.26%,0.05%,12.39%,47.58%


C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\1418307944.py:90: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-0.67%,16.39%,-1.82%,nan%,-13.50%,-12.25%,9.50%,7.78%,nan%,2.35%,8.62%,-12.95%,3.82%,-3.10%,3.40%,16.50%,30.32%,nan%,nan%,nan%,nan%,nan%,nan%
2020,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-3.44%,15.84%,nan%,5.61%,-5.23%,-1.60%,-1.36%,1.19%,6.70%,-1.50%,30.06%,19.74%,nan%,4.75%,11.14%,-9.89%,9.78%,-17.75%,3.85%,1.26%,-3.65%,-1.46%,nan%,6.19%,1.05%,7.34%,-2.26%,nan%,14.53%,-1.37%,24.99%,2.91%,4.38%,-1.81%,8.02%,7.37%,42.79%
2021,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,nan%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-11.64%,17.17%,nan%,-7.85%,-11.63%,15.76%,16.71%,nan%,17.86%,9.80%,-2.10%,-0.45%,22.55%,-13.82%,-2.22%,-8.09%,11.71%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,3.68%,-3.89%,-6.60%,4.84%,24.59%,-1.19%,nan%,-1.97%,-2.21%,-5.61%,0.50%,nan%,-14.75%,10.72%,8.96%,-7.52%,-17.77%,-4.97%,2.22%,5.13%,-11.67%,nan%,-5.29%,0.82%,11.44%,2.97%,-8.19%,1.18%,11.26%,-8.70%,-2.00%,nan%,29.52%,-3.37%,9.27%,10.31%,nan%,8.00%,-2.19%,nan%,nan%
Avg,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,15.40%,-3.89%,13.22%,-1.93%,-6.77%,9.48%,13.67%,-1.28%,-6.01%,-6.28%,5.44%,1.19%,-5.30%,-0.81%,18.26%,9.64%,-17.77%,5.88%,5.62%,2.38%,-1.04%,2.40%,-7.19%,-3.10%,2.30%,5.25%,-8.19%,3.24%,6.98%,-4.77%,-0.15%,-3.10%,15.82%,3.92%,21.53%,6.61%,4.38%,3.09%,2.91%,7.37%,42.79%
Median,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,15.40%,-3.89%,13.22%,-1.93%,-3.44%,13.80%,13.67%,-1.97%,-5.23%,-5.61%,0.50%,1.19%,-7.85%,-1.50%,15.76%,16.71%,-17.77%,4.75%,6.01%,1.52%,-1.13%,2.40%,-9.39%,-0.70%,2.92%,5.38%,-8.19%,2.35%,8.62%,-8.70%,-2.00%,-3.10%,14.53%,-1.37%,24.99%,6.61%,4.38%,3.09%,2.91%,7.37%,42.79%
Max,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,-3.89%,33.05%,4.84%,24.59%,15.84%,13.67%,5.61%,-2.21%,-1.60%,17.17%,1.19%,6.70%,10.72%,30.06%,19.74%,-17.77%,17.86%,11.14%,16.39%,9.78%,22.55%,3.85%,1.26%,11.44%,11.71%,-8.19%,6.19%,11.26%,7.34%,3.82%,-3.10%,29.52%,16.50%,30.32%,10.31%,4.38%,8.00%,8.02%,7.37%,42.79%
Min,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,3.68%,-3.89%,-6.60%,-8.71%,-41.47%,-1.19%,13.67%,-7.47%,-10.59%,-11.64%,-1.36%,1.19%,-14.75%,-11.63%,8.96%,-7.52%,-17.77%,-4.97%,-0.67%,-9.89%,-11.67%,-17.75%,-13.82%,-12.25%,-8.09%,-1.46%,-8.19%,1.18%,1.05%,-12.95%,-2.26%,-3.10%,3.40%,-3.37%,9.27%,2.91%,4.38%,-1.81%,-2.19%,7.37%,42.79%


In [5]:
import ccxt
import pandas as pd
from IPython.display import display, HTML
from datetime import timedelta

# Function to fetch ETH/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    # Create unique "week number" format: MonthLetter-MonthNumber-WeekOfMonth
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)
    
    return df

# Function to filter data for the 18 months after each halving event
def filter_halving_data(df):
    halving_dates = ['2012-11-28', '2016-07-10', '2020-05-12', '2024-04-20']
    halving_dates = pd.to_datetime(halving_dates)

    # Filter the DataFrame to only include data from 18 months after each halving date
    filtered_df = pd.DataFrame()
    for halving_date in halving_dates:
        start_date = halving_date
        end_date = start_date + timedelta(weeks=18*4)  # 18 months = 18 * 4 weeks
        filtered_df = pd.concat([filtered_df, df[(df.index >= start_date) & (df.index <= end_date)]])

    return filtered_df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to add the halving week number and halving count columns
def add_halving_columns(df):
    halving_dates = pd.to_datetime(['2012-11-28', '2016-07-10', '2020-05-12', '2024-04-20'])
    halving_counts = [1, 2, 3, 4]
    
    # Create new columns to store the week after halving and halving count
    df['week_after_halving'] = None
    df['halving_count'] = None

    # Iterate over each halving event
    for i, halving_date in enumerate(halving_dates):
        # Calculate the weeks since each halving event
        df.loc[df.index >= halving_date, 'week_after_halving'] = ((df.index[df.index >= halving_date] - halving_date).days // 7) + 1
        df.loc[df.index >= halving_date, 'halving_count'] = halving_counts[i]
    
    return df

# Function to create and style the pivot table with positive percentages in green
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('week_after_halving')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='halving_count', columns='week_after_halving', aggfunc='mean')

    # Calculate mean, median, max, and min for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)
    max_value = pivot_table.max(axis=0)
    min_value = pivot_table.min(axis=0)

    # Add the average, median, max, and min as the last rows
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value
    pivot_table.loc['Max'] = max_value
    pivot_table.loc['Min'] = min_value

    # Function to color positive values green and negative values red
    def color_positive_negative(val):
        if isinstance(val, (int, float)):
            if val > 0:
                return 'background-color: lightgreen; color: white;'  # Green for positive values
            elif val < 0:
                return 'background-color: pink; color: white;'  # Red for negative values
        return ''  # No change for non-numeric values

    # Style the table
    styled_table = pivot_table.style \
        .format('{:.2f}%') \
        .applymap(color_positive_negative)  # Apply the coloring function to the table
    
    # Set additional styles
    styled_table.set_table_styles([ 
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%'), ('display', 'block'), ('overflow-x', 'auto'), ('position', 'relative')]},  # Set relative position for the table container
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
        
        # Add a line below the "Avg" row to separate it from the rest of the table
        {'selector': 'tr:nth-last-child(5)', 'props': [('border-bottom', '2px solid black')]},  # Row for "Avg"
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table

# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ETH/USDT'
    ETH_data = fetch_data(symbol)

    # Filter data for 18 months after each halving
    ETH_data = filter_halving_data(ETH_data)

    # Add halving week and halving count columns
    ETH_data = add_halving_columns(ETH_data)

    # Calculate indicators
    ETH_data = calculate_max_drawdown(ETH_data)
    ETH_data = calculate_high_to_open_change(ETH_data)
    ETH_data = calculate_weekly_return(ETH_data)

    # Create and display the pivot tables
    styled_table_max_drawdown = create_pivot_table(ETH_data, 'max_drawdown')
    display_table("ETH/USDT Weekly Max Drawdown (Week After Halving)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(ETH_data, 'high_to_open_change')
    display_table("ETH/USDT Weekly Open to High % Change (Week After Halving)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(ETH_data, 'weekly_return')
    display_table("ETH/USDT Weekly Returns (Open to Close, Week After Halving)", styled_table_weekly_return)

# Run the main function
main()



C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\612253741.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-7.68%,-51.79%,-8.05%,-30.58%,-34.58%,-2.11%,-2.84%,-6.83%,-7.87%,-15.65%,-7.35%,-9.92%,-4.80%,-0.10%,-1.44%
3,-7.40%,-1.50%,-2.83%,-7.51%,-5.81%,-5.28%,-1.61%,-0.27%,-5.25%,-2.12%,-1.66%,-2.58%,-6.23%,-12.38%,-5.30%,-28.10%,-8.46%,-3.12%,-15.56%,-6.53%,-5.72%,-3.47%,-3.40%,-8.02%,-6.67%,-4.67%,-0.58%,-14.11%,-2.83%,-11.83%,-2.57%,-13.61%,-0.59%,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-24.20%,-1.10%,-11.89%,-13.83%,-9.81%,-1.80%,-4.43%,-3.94%,-10.80%,-5.70%,-2.50%,-23.93%,-8.69%,-20.33%,-9.07%
4,-2.57%,-13.68%,-8.24%,-2.30%,-0.77%,-3.24%,-4.79%,-9.29%,-7.43%,-5.29%,-18.27%,-3.69%,-0.37%,-12.68%,-19.58%,-21.49%,-1.77%,-2.91%,-12.86%,-11.34%,-1.07%,-2.75%,-1.61%,-13.08%,-4.48%,-1.03%,-13.26%,-3.86%,-4.07%,-5.30%,-1.51%,-3.25%,-5.60%,-12.37%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,-4.98%,-7.59%,-5.54%,-4.90%,-3.29%,-4.26%,-3.20%,-4.78%,-6.34%,-3.70%,-9.96%,-3.13%,-3.30%,-12.53%,-12.44%,-24.79%,-5.12%,-3.01%,-14.21%,-8.94%,-3.39%,-3.11%,-2.50%,-10.55%,-5.57%,-2.85%,-6.92%,-8.99%,-3.45%,-8.56%,-2.04%,-8.43%,-3.10%,-10.70%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-15.94%,-26.44%,-9.97%,-22.20%,-22.20%,-1.95%,-3.63%,-5.38%,-9.34%,-10.68%,-4.92%,-16.93%,-6.75%,-10.21%,-5.26%
Median,-4.98%,-7.59%,-5.54%,-4.90%,-3.29%,-4.26%,-3.20%,-4.78%,-6.34%,-3.70%,-9.96%,-3.13%,-3.30%,-12.53%,-12.44%,-24.79%,-5.12%,-3.01%,-14.21%,-8.94%,-3.39%,-3.11%,-2.50%,-10.55%,-5.57%,-2.85%,-6.92%,-8.99%,-3.45%,-8.56%,-2.04%,-8.43%,-3.10%,-10.70%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-15.94%,-26.44%,-9.97%,-22.20%,-22.20%,-1.95%,-3.63%,-5.38%,-9.34%,-10.68%,-4.92%,-16.93%,-6.75%,-10.21%,-5.26%
Max,-2.57%,-1.50%,-2.83%,-2.30%,-0.77%,-3.24%,-1.61%,-0.27%,-5.25%,-2.12%,-1.66%,-2.58%,-0.37%,-12.38%,-5.30%,-21.49%,-1.77%,-2.91%,-12.86%,-6.53%,-1.07%,-2.75%,-1.61%,-8.02%,-4.48%,-1.03%,-0.58%,-3.86%,-2.83%,-5.30%,-1.51%,-3.25%,-0.59%,-9.03%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-7.68%,-1.10%,-8.05%,-13.83%,-9.81%,-1.80%,-2.84%,-3.94%,-7.87%,-5.70%,-2.50%,-9.92%,-4.80%,-0.10%,-1.44%
Min,-7.40%,-13.68%,-8.24%,-7.51%,-5.81%,-5.28%,-4.79%,-9.29%,-7.43%,-5.29%,-18.27%,-3.69%,-6.23%,-12.68%,-19.58%,-28.10%,-8.46%,-3.12%,-15.56%,-11.34%,-5.72%,-3.47%,-3.40%,-13.08%,-6.67%,-4.67%,-13.26%,-14.11%,-4.07%,-11.83%,-2.57%,-13.61%,-5.60%,-12.37%,-27.05%,-15.43%,-13.30%,-3.24%,-2.95%,-8.02%,-33.12%,-0.62%,-3.56%,-7.38%,-13.81%,-0.55%,-7.02%,-9.51%,-8.08%,-0.72%,-0.00%,-14.80%,-51.74%,-0.80%,-4.79%,-16.80%,-18.68%,-24.20%,-51.79%,-11.89%,-30.58%,-34.58%,-2.11%,-4.43%,-6.83%,-10.80%,-15.65%,-7.35%,-23.93%,-8.69%,-20.33%,-9.07%


C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\612253741.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,3.67%,16.39%,13.29%,1.55%,7.82%,18.91%,10.99%,4.79%,13.77%,4.89%,5.52%,3.61%,11.45%,22.88%,36.41%
3,4.85%,23.56%,9.60%,2.33%,2.40%,9.30%,3.48%,9.38%,1.06%,33.30%,33.71%,9.28%,13.99%,2.86%,10.03%,13.96%,10.65%,7.56%,1.55%,3.60%,7.40%,5.51%,11.27%,1.30%,18.05%,5.22%,29.64%,11.24%,10.30%,0.26%,14.46%,12.39%,47.58%,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,0.74%,20.57%,3.72%,1.27%,16.27%,23.19%,24.83%,10.60%,0.84%,4.32%,24.95%,0.50%,7.99%,0.58%,14.01%
4,6.66%,0.72%,2.71%,7.43%,28.59%,3.93%,2.82%,0.20%,0.39%,1.95%,2.52%,11.52%,9.30%,0.76%,3.87%,1.31%,8.79%,7.96%,0.58%,5.74%,7.29%,13.66%,5.72%,0.22%,3.32%,11.75%,0.82%,8.55%,32.18%,8.15%,13.70%,11.47%,10.25%,0.05%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,5.76%,12.14%,6.16%,4.88%,15.50%,6.62%,3.15%,4.79%,0.72%,17.63%,18.11%,10.40%,11.64%,1.81%,6.95%,7.64%,9.72%,7.76%,1.07%,4.67%,7.34%,9.59%,8.49%,0.76%,10.68%,8.49%,15.23%,9.90%,21.24%,4.20%,14.08%,11.93%,28.92%,18.93%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,2.21%,18.48%,8.51%,1.41%,12.05%,21.05%,17.91%,7.69%,7.30%,4.60%,15.24%,2.05%,9.72%,11.73%,25.21%
Median,5.76%,12.14%,6.16%,4.88%,15.50%,6.62%,3.15%,4.79%,0.72%,17.63%,18.11%,10.40%,11.64%,1.81%,6.95%,7.64%,9.72%,7.76%,1.07%,4.67%,7.34%,9.59%,8.49%,0.76%,10.68%,8.49%,15.23%,9.90%,21.24%,4.20%,14.08%,11.93%,28.92%,18.93%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,2.21%,18.48%,8.51%,1.41%,12.05%,21.05%,17.91%,7.69%,7.30%,4.60%,15.24%,2.05%,9.72%,11.73%,25.21%
Max,6.66%,23.56%,9.60%,7.43%,28.59%,9.30%,3.48%,9.38%,1.06%,33.30%,33.71%,11.52%,13.99%,2.86%,10.03%,13.96%,10.65%,7.96%,1.55%,5.74%,7.40%,13.66%,11.27%,1.30%,18.05%,11.75%,29.64%,11.24%,32.18%,8.15%,14.46%,12.39%,47.58%,37.82%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,3.67%,20.57%,13.29%,1.55%,16.27%,23.19%,24.83%,10.60%,13.77%,4.89%,24.95%,3.61%,11.45%,22.88%,36.41%
Min,4.85%,0.72%,2.71%,2.33%,2.40%,3.93%,2.82%,0.20%,0.39%,1.95%,2.52%,9.28%,9.30%,0.76%,3.87%,1.31%,8.79%,7.56%,0.58%,3.60%,7.29%,5.51%,5.72%,0.22%,3.32%,5.22%,0.82%,8.55%,10.30%,0.26%,13.70%,11.47%,10.25%,0.05%,3.49%,16.70%,5.95%,34.14%,16.15%,13.46%,0.14%,22.24%,12.62%,2.36%,1.36%,27.20%,5.99%,18.45%,18.29%,28.64%,35.07%,11.44%,0.15%,38.79%,21.21%,5.00%,5.21%,0.74%,16.39%,3.72%,1.27%,7.82%,18.91%,10.99%,4.79%,0.84%,4.32%,5.52%,0.50%,7.99%,0.58%,14.01%


C:\Users\nicol\AppData\Local\Temp\ipykernel_26000\612253741.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-0.67%,16.39%,-1.82%,-13.50%,-12.25%,9.50%,7.78%,2.35%,8.62%,-12.95%,3.82%,-3.10%,3.40%,16.50%,30.32%
3,-3.44%,15.84%,5.61%,-5.23%,-1.60%,-1.36%,1.19%,6.70%,-1.50%,30.06%,19.74%,4.75%,11.14%,-9.89%,9.78%,-17.75%,3.85%,1.26%,-3.65%,-1.46%,6.19%,1.05%,7.34%,-2.26%,14.53%,-1.37%,24.99%,2.91%,4.38%,-1.81%,8.02%,7.37%,42.79%,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-11.64%,17.17%,-7.85%,-11.63%,15.76%,16.71%,17.86%,9.80%,-2.10%,-0.45%,22.55%,-13.82%,-2.22%,-8.09%,11.71%
4,3.68%,-3.89%,-6.60%,4.84%,24.59%,-1.19%,-1.97%,-2.21%,-5.61%,0.50%,-14.75%,10.72%,8.96%,-7.52%,-17.77%,-4.97%,2.22%,5.13%,-11.67%,-5.29%,0.82%,11.44%,2.97%,-8.19%,1.18%,11.26%,-8.70%,-2.00%,29.52%,-3.37%,9.27%,10.31%,8.00%,-2.00%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,0.12%,5.98%,-0.49%,-0.19%,11.50%,-1.28%,-0.39%,2.24%,-3.56%,15.28%,2.50%,7.73%,10.05%,-8.70%,-4.00%,-11.36%,3.03%,3.19%,-7.66%,-3.38%,3.51%,6.25%,5.15%,-5.22%,7.85%,4.95%,8.14%,0.45%,16.95%,-2.59%,8.64%,8.84%,25.40%,13.10%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-6.16%,16.78%,-4.83%,-12.56%,1.75%,13.10%,12.82%,6.07%,3.26%,-6.70%,13.18%,-8.46%,0.59%,4.20%,21.02%
Median,0.12%,5.98%,-0.49%,-0.19%,11.50%,-1.28%,-0.39%,2.24%,-3.56%,15.28%,2.50%,7.73%,10.05%,-8.70%,-4.00%,-11.36%,3.03%,3.19%,-7.66%,-3.38%,3.51%,6.25%,5.15%,-5.22%,7.85%,4.95%,8.14%,0.45%,16.95%,-2.59%,8.64%,8.84%,25.40%,13.10%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-6.16%,16.78%,-4.83%,-12.56%,1.75%,13.10%,12.82%,6.07%,3.26%,-6.70%,13.18%,-8.46%,0.59%,4.20%,21.02%
Max,3.68%,15.84%,5.61%,4.84%,24.59%,-1.19%,1.19%,6.70%,-1.50%,30.06%,19.74%,10.72%,11.14%,-7.52%,9.78%,-4.97%,3.85%,5.13%,-3.65%,-1.46%,6.19%,11.44%,7.34%,-2.26%,14.53%,11.26%,24.99%,2.91%,29.52%,-1.81%,9.27%,10.31%,42.79%,28.20%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-0.67%,17.17%,-1.82%,-11.63%,15.76%,16.71%,17.86%,9.80%,8.62%,-0.45%,22.55%,-3.10%,3.40%,16.50%,30.32%
Min,-3.44%,-3.89%,-6.60%,-5.23%,-1.60%,-1.36%,-1.97%,-2.21%,-5.61%,0.50%,-14.75%,4.75%,8.96%,-9.89%,-17.77%,-17.75%,2.22%,1.26%,-11.67%,-5.29%,0.82%,1.05%,2.97%,-8.19%,1.18%,-1.37%,-8.70%,-2.00%,4.38%,-3.37%,8.02%,7.37%,8.00%,-2.00%,-1.74%,12.98%,-5.72%,22.79%,11.73%,7.42%,-26.60%,21.67%,7.05%,-3.52%,-5.40%,23.03%,3.65%,3.91%,3.79%,27.11%,33.05%,-8.71%,-41.47%,13.80%,13.67%,-7.47%,-10.59%,-11.64%,16.39%,-7.85%,-13.50%,-12.25%,9.50%,7.78%,2.35%,-2.10%,-12.95%,3.82%,-13.82%,-2.22%,-8.09%,11.71%
